In [1]:
%AddDeps com.databricks spark-csv_2.10 1.3.0 --transitive

:: loading settings :: url = jar:file:/opt/spark-1.5.1-bin-hadoop2.3/lib/spark-assembly-1.5.1-hadoop2.3.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
:: resolving dependencies :: com.ibm.spark#spark-kernel;working
	confs: [default]
	found com.databricks#spark-csv_2.10;1.3.0 in central
	found org.apache.commons#commons-csv;1.1 in central
	found com.univocity#univocity-parsers;1.5.1 in central
:: resolution report :: resolve 189ms :: artifacts dl 8ms
	:: modules in use:
	com.databricks#spark-csv_2.10;1.3.0 from central in [default]
	com.univocity#univocity-parsers;1.5.1 from central in [default]
	org.apache.commons#commons-csv;1.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |  

In [2]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._ 

val sqlContext = new SQLContext(sc)

//Data is not aggregated
val co2 = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load("Co2all.csv")

//Add a column that has Year as a true date type
val convertDate = udf {
    (year:Int) => year+ "-01-01 " +year + " 00:00 UTC"
}
val co2dates = co2.withColumn("year_as_date", to_date(convertDate(co2("Year"))) )

//Data is aggregated by Spark
val co2agg = co2.groupBy("CO2 per capita").agg( avg("value") as "Mean Co2")

#Average CO2 emissions per Country
Here, Spark does the aggegation.  Use the mouse wheel to zoom the map, click to pan.

In [3]:
%%brunel data('co2agg') map(low) x(CO2_per_capita) color(Mean_Co2) tooltip(#all):: width=800, height=500

#Carbon emissions over time
Click on a country in the map to see its CO2 emmissions over time

In [4]:
%%brunel data('co2dates') 
        map(low) bar x(CO2_per_capita) color(value) mean(value) interaction(select) at(0,0,100,50) tooltip(#all) |
        x(year_as_date) y(value) line label(CO2_per_capita) interaction(filter) at(0,50, 100,100) 
:: width=900, height=600